## Gluing nodes and de Bruijn graphs

In [1]:
def Composition(DnaString:str, k:int):
    return [DnaString[i:k+i] for i in range(len(DnaString) - k + 1)]


def CompositionNoRepeats(DnaString:str, k:int):
    return list({
        DnaString[i:i+k]: [] for i in range(len(DnaString)-k+1)
    }.keys())

In [2]:
def Prefix(String:str):
    return String[:-1]


def Suffix(String:str):
    return String[1:]

We create Graph from common 2-mers acting as ***nodes***, where our 3-mers act like ***edges***. After that we start gluing common nodes together, until we minimalise the Graph. 

**This is De Bruijn Graph of TAATGCCATGGGATGTT**

#### Hamiltonian Path Problem: 
Construct a Hamiltonian path in a graph

In [3]:
def DeBruijnGraphFromString(
    DnaString:str, k:int
):
    kMersList = Composition(DnaString, k)
    PrefixDict = {}    
    
    for kMer in kMersList:
        prefix = Prefix(kMer)
        suffix = Suffix(kMer)

        if prefix not in PrefixDict:
            PrefixDict[prefix] = list()
        PrefixDict[prefix].append(suffix)

    for key, value in sorted(PrefixDict.items()):
        print(f"{key}:", *value)

In [4]:
DeBruijnGraphFromString(
    DnaString='TAATGGGATGCCATGTT',
    k=4
)

AAT: ATG
ATG: TGG TGC TGT
CAT: ATG
CCA: CAT
GAT: ATG
GCC: CCA
GGA: GAT
GGG: GGA
TAA: AAT
TGC: GCC
TGG: GGG
TGT: GTT


Solving the String Reconstruction Problem reduces to finding a path in the de Bruijn graph that visits every edge exactly once. Such a path is called an Eulerian Path in honor of the great mathematician Leonhard Euler (pronounced “oiler").

Hamiltonian Path - graph that visits every node exactly once

Eulerian Path - graph that visits every edge exactly once

#### Eulerian Path Problem:
Construct an Eulerian path in a graph.

In [5]:
def DeBruijnGraphFromKMers(
    kMersList:list
):
    PrefixDict = {}    
    
    for kMer in kMersList:
        prefix = Prefix(kMer)
        suffix = Suffix(kMer)

        if prefix not in PrefixDict:
            PrefixDict[prefix] = list()
        PrefixDict[prefix].append(suffix)

    for key, value in sorted(PrefixDict.items()):
        print(f"{key}:", *value)

In [ ]:
DeBruijnGraphFromKMers(
    kMersList=[kMer for kMer in input().split()]
)

AGG: GGG
CAG: AGG AGG
GAG: AGG
GGA: GAG
GGG: GGG GGA


Thus we can create scheme of k-mers and edges knowing only list of k-mers, and the same way we know, how to create such scheme from genome string. That means we can restore genome from k-mers list via De Bruijn Graph

## Eulerian Cycle
Cycle in graph, that goes through every edge exactly once

In every Eulerian Cycle for every node number of incoming edges equals to number of outcoming edges

Balanced Graph: ***indegree=outdegree***. Meaning that every Eulerian Graph is balanced

#### Euler’s Theorem:
**Every balanced, strongly connected directed graph is Eulerian.**

When travelling through balanced Graph, we can stuck and not complete full path only if we started in a wrong place - not at the starting point!

So we can start at random point of a graph, try to comlete all the way, if we get stuck we can start again from last node of creatd path, where there are still unexplored edges! If we repeat such process, we will somewhen find a proper starting point and complete an Eulerian cycle!

In [189]:
from random import choice

def EulerianCycle(Graph):
    Edge = choice(list(Graph.keys()))
    remainingEdges = [ i for j in Graph.values() for i in j ]
    remainingEdges.remove(Edge)
    Cycle = [Edge]
    rejectesEdges = list()

    print(f"Starting:\n{Edge} {remainingEdges}")

    while remainingEdges:
        lastEdge = Cycle[-1]
        print(Graph[lastEdge], remainingEdges)
        print()

        if list(set(Graph[lastEdge]) & set(remainingEdges)):
            currentEdge = choice(list(set(Graph[lastEdge]) & set(remainingEdges)))
            Cycle.append(currentEdge)
            lastEdge = currentEdge
            remainingEdges.remove(currentEdge)
            print(lastEdge, remainingEdges)

        else:   
            # здесь нужен откат назад
            # не работает следующее - мы хотим находить ребро , у которого было больше  одного пути
            for i in Cycle[::-1]:
                print(i)
                if len(Graph[i]) > 1:
                    print(Graph[i])
                    break
            
            break



    Cycle.append(Cycle[0])
    return Cycle

    # return Cycle


# def ExtendCycle(Graph, Node, Sequence):
    # 

In [190]:
exampleGraph = {
    0: [3], 1: [0],
    2: [1, 6], 3: [2],
    4: [2], 5: [4], 6: [5, 8], 7: [9],
    8: [7], 9: [6]
}

In [192]:
EulerianCycle(Graph=exampleGraph)

Starting:
0 [3, 1, 6, 2, 2, 4, 5, 8, 9, 7, 6]
[3] [3, 1, 6, 2, 2, 4, 5, 8, 9, 7, 6]

3 [1, 6, 2, 2, 4, 5, 8, 9, 7, 6]
[2] [1, 6, 2, 2, 4, 5, 8, 9, 7, 6]

2 [1, 6, 2, 4, 5, 8, 9, 7, 6]
[1, 6] [1, 6, 2, 4, 5, 8, 9, 7, 6]

6 [1, 2, 4, 5, 8, 9, 7, 6]
[5, 8] [1, 2, 4, 5, 8, 9, 7, 6]

5 [1, 2, 4, 8, 9, 7, 6]
[4] [1, 2, 4, 8, 9, 7, 6]

4 [1, 2, 8, 9, 7, 6]
[2] [1, 2, 8, 9, 7, 6]

2 [1, 8, 9, 7, 6]
[1, 6] [1, 8, 9, 7, 6]

1 [8, 9, 7, 6]
[0] [8, 9, 7, 6]

1
2
[1, 6]


[0, 3, 2, 6, 5, 4, 2, 1, 0]

На данный момент работает собирание цикла
Надо теперь научиться откатываться на несколько шагов назад

In [186]:
any(x in [3, 0, 1, 2]  for x in [5, 8])

False